<a href="https://colab.research.google.com/github/venkatab144/UTS_ML2019_ASS2_12954525/blob/master/ML_A2_12954525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Discuss the Algorithm; Define Input/Output W-100

# Exploration

Identify Challenges;
Design Data Structures/Plan Data Models and Tests
W-300

# Methodology

Implement Algorithms;
Build and Train Data Models
W-100 (comments not included)

# Evaluation

Report Execution on Data;
Perform and Report Testing;
Perform Efficiency Analysis;
Do Possible Compartive Study;
W-200;

# Conclusion

Discuss Reflections;
Propose Possible Improvements
W-100

# Ethical

Discuss the social/ethical aspect of the potential application of the algorithm. You can adopt one or more ethical models, such as the utilitarian approach or the Kantian duty-based approach. Should anticipate the potential misuses of the technique.

# Video Pitch

Highlights Challenges and Effort